# lesson5-2 CTR预估算法与基于流行度的推荐

## Thinking1 在CTR点击率预估中，使用GBDT+LR的原理是什么？

通过将GBDT的特征进行组合，然后传给线性分类器(LR)去使用。    
GBDT(由多颗cart决策树组成)在训练好使用的时候，输出的并不是最终的二分类概率值，而是把模型中每个树的概率预测值所属的叶子节点标记为1，通过这种方式的来构造新的特征。然后将此特征进行处理后再给到LR去进行二分类。  

## Thinking2 Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）


Wide推荐：系统通过获得用户的购物数据，包括点击过哪些商品，购买过哪些商品。然后通过OneHot编码转换为离散特征，并通过embedding操作来降维；连续特征可以处理后使用；同时可以通过对离散特征进行特征组合生成新的特征。最后将embedding后的离散特征和本身的连续特征以及组合特征生成的新特征，统一进行特征处理和学习。该方法可解释性强，但是需要人工的进行特征操作。    
Deep推荐：通过深度学习学习出一些向量，这些向量是隐性特征，一般没有可解释性。    
Wide & Deep：通过将wide和deep结合，把两个模型组合ensemble来进行预测，从而使得模型既具有wide的记忆能力，又具有deep的泛化能力。

## Thinking3 在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？

两个模型并行的结合方式，代表有DeepFM，Wide&Deep   
两个模型串行的结合方式，代表有NFM   
两者的区别在于DeepFm可以将FM和DNN分开计算，可以并行的进行处理，而NFM由于需要将FM的结果作为DNN的输入，只能串行的进行处理

## Thinking4 GBDT和随机森林都是基于树的算法，它们有什么区别？


- GBDT是boosting算法     
该方法的每一轮学习，都是根据当前多个树模型组合的值和真实值的差距来进行拟合学习。当采用平方误差时，体现出来的是不断的对残差进行拟合。但是由于只有训练好一个弱分类器，才可以进行后续分类器的训练，各个分类器有前后的关系，所以该方法只可以串行的进行训练。    
- 随机森林是bagging算法     
使用时会随机的选择数据进行训练很多个决策树，然后将多个模型共同的去进行最终结果的预测。由于多个决策树是各自单独训练的，所以随机森林可以并行的去计算训练。


## Thinking5 item流行度在推荐系统中有怎样的应用


- 最常见的是将榜单中热度的内容推荐给用户   
- 基于流行度的推荐是围绕流行度计算产生的推荐模型  
- 还可以根据流行度来推荐商品的算法，也就是什么内容吸引用户，就给用户推荐什么内容，来解决冷启动问题    
- 在推荐系统中相同相似度下，考虑降低流行度的权重，更能代表用户的兴趣。

## Action1 使用Wide&Deep模型对movielens进行评分预测


In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat,get_feature_names

In [3]:
# 加载数据
data=pd.read_csv('movielens_sample.txt')
sparse_features=['movie_id','user_id','gender','age','occupation','zip']
target=['rating']

In [4]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,107,12,4,968035345,Ed Wood (1994),Comedy|Drama,0,2,4,35
1,123,169,3,966536874,Patriot Games (1992),Action|Thriller,1,1,4,118
2,12,6,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,0,2,13,99
3,21,112,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,1,1,18,55
4,187,45,5,957782527,"Apartment, The (1960)",Comedy|Drama,1,5,19,41
...,...,...,...,...,...,...,...,...,...,...
195,46,176,3,974840560,Screwed (2000),Comedy,1,2,11,48
196,131,89,3,965855033,Fire Down Below (1997),Action|Drama|Thriller,1,1,11,113
197,4,125,3,976730191,Desperately Seeking Susan (1985),Comedy|Romance,0,1,13,83
198,181,15,4,958503395,Clear and Present Danger (1994),Action|Adventure|Thriller,1,2,0,106


In [5]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns=[SparseFeat(feature,data[feature].nunique()) for feature in sparse_features]
linear_feature_columns=fixlen_feature_columns
dnn_feature_columns=fixlen_feature_columns
feature_names=get_feature_names(linear_feature_columns+dnn_feature_columns)
fixlen_feature_columns

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[SparseFeat(name='movie_id', vocabulary_size=187, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000002B326E213C8>, embedding_name='movie_id', group_name='default_group', trainable=True),
 SparseFeat(name='user_id', vocabulary_size=193, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000002B326DD7848>, embedding_name='user_id', group_name='default_group', trainable=True),
 SparseFeat(name='gender', vocabulary_size=2, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000002B326DD78C8>, embedding_name='gender', group_name='default_group', trainable=True),
 SparseFeat(name='age', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000002

In [6]:
# 将数据集切分成训练集和测试集
train,test=train_test_split(data,test_size=0.2)
train_model_input={name:train[name].values for name in feature_names}
test_model_input={name:test[name].values for name in feature_names}
test_model_input

{'movie_id': array([142,   8,   9, 146,  82, 170,  99, 156, 149,  25,  85,  49, 123,
         31,  51, 169, 182,  30, 102,  19, 184, 165,  29, 107, 132,  11,
         48, 112, 168,  73, 173, 149, 176, 134,  27,  18,  66,  67, 137,
        126], dtype=int64),
 'user_id': array([  5, 150,  45,  99,  97, 178,  56,  24, 109,   9,  84,  76, 147,
        124,  79, 123, 106, 134,  48,  27,  61, 156, 108,  59, 125,  13,
         74,  21, 192, 166,  10, 142,  46, 112,  39, 158, 183,  69, 175,
         94], dtype=int64),
 'gender': array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]),
 'age': array([2, 6, 2, 1, 3, 2, 2, 5, 2, 2, 0, 5, 5, 3, 5, 1, 2, 2, 2, 5, 2, 5,
        1, 2, 2, 2, 1, 1, 2, 2, 3, 2, 2, 3, 1, 3, 1, 4, 4, 1], dtype=int64),
 'occupation': array([11,  7,  0,  0, 19,  6, 19,  1, 11, 11,  9, 15,  6,  7, 11,  4, 19,
        16, 11,  0,  0, 17,  4,  3,  4,  0,  4, 18,  6,  7,  7,  0, 11,  1,
         1,

In [7]:
# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=3, verbose=True, validation_split=0.2, )

Epoch 1/3


d:\Users\rexcase\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 256ms/step - loss: 14.1014 - mse: 14.1014 - val_loss: 13.6045 - val_mse: 13.6045
Epoch 2/3
1/1 [==============================] - 0s 38ms/step - loss: 13.9637 - mse: 13.9637 - val_loss: 13.4839 - val_mse: 13.4839
Epoch 3/3
1/1 [==============================] - 0s 35ms/step - loss: 13.8166 - mse: 13.8166 - val_loss: 13.3552 - val_mse: 13.3552


In [9]:
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)

In [10]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 3.7711934450515794
